# Variables

In [1]:
# surpress warning
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

sess = tf.Session()

/Users/larrysu/.pyenv/versions/p3lab/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
tf.reset_default_graph()
sess = tf.Session()
v1 = tf.get_variable("my_var", dtype=tf.int32, initializer=tf.constant([[1,2,3],[4,5,6]]))
# v1 = tf.get_variable("my_var", [1,2,3], initializer=tf.zeros_initializer)
# v1 = tf.get_variable("my_var", [1,2,3])

with sess.as_default():
    init = tf.global_variables_initializer()
    sess.run(init)
    v1_value = v1.eval()

print(v1_value)

[[1 2 3]
 [4 5 6]]


## Variable collections

- tf.GraphKeys.GLOBAL_VARIABLES : shared across devices
- tf.GraphKeys.TRAINABLE_VARIABLES : will calculate gradients

```
with tf.device("/device:GPU:1"):
  v = tf.get_variable("v", [1])

cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]}
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
  v = tf.get_variable("v", shape=[20, 20])  # this variable is placed
                                            # in the parameter server
                                            # by the replica_device_setter

```

In [17]:
tf.reset_default_graph()
sess = tf.Session()

v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
  w = v.read_value()  # w is guaranteed to reflect v's value after the
                      # assign_add operation.

sess.run(tf.global_variables_initializer())
print(sess.run(w))

1.0


## Sharing Variables

```
with tf.variable_scope("model"):
  output1 = my_image_filter(input1)
with tf.variable_scope("model", reuse=True):
  output2 = my_image_filter(input2)

```